# Bloom filter

A Bloom filter is a space-efficient probabilistic data structure, conceived by Burton Howard Bloom in 1970, that is used to test whether an element is a member of a set. False positive matches are possible, but false negatives are not – in other words, a query returns either "possibly in set" or "definitely not in set". Elements can be added to the set, but not removed (though this can be addressed with a "counting" filter); the more elements that are added to the set, the larger the probability of false positives.

An empty Bloom filter is a bit array of $m$ bits, all set to 0. There must also be $k$ different hash functions defined, each of which maps or hashes some set element to one of the m array positions, generating a uniform random distribution. Typically, $k$ is a constant, much smaller than $m$, which is proportional to the number of elements to be added; the precise choice of $k$ and the constant of proportionality of $m$ are determined by the intended false positive rate of the filter.

To add an element, feed it to each of the $k$ hash functions to get $k$ array positions. Set the bits at all these positions to 1.

To query for an element (test whether it is in the set), feed it to each of the $k$ hash functions to get $k$ array positions. If any of the bits at these positions is 0, the element is definitely not in the set – if it were, then all the bits would have been set to 1 when it was inserted. If all are 1, then either the element is in the set, or the bits have by chance been set to 1 during the insertion of other elements, resulting in a false positive. In a simple Bloom filter, there is no way to distinguish between the two cases, but more advanced techniques can address this problem.

*Shamelessly copied from [Wikipedia article](https://en.wikipedia.org/wiki/Bloom_filter).*

## Analysis

Probability that a given bit of the bloom filter has been set to 1 after $n$ insertions

$$ P(\text{bit}=1) = 1 - \left(1 - \frac{1}{m}\right)^{kn}. $$

Probability of false positives is

$$ \left(1 - \left(1 - \frac{1}{m} \right) ^ {kn} \right) ^ k \approx \left(1 - e ^ {- \frac{kn}{m}} \right) ^ k $$

## Optimal number of hash functions

Given a number of bits $m$ and a number of values to be inserted $n$, we can compute optimal number of hash functions $k$

$$
\begin{align*}
\dfrac{\mathrm{d}}{\mathrm{d}k} \left[ \left( 1 - e ^ {- \frac{kn}{m}} \right) ^ k \right] &= \dfrac{\mathrm{d}}{\mathrm{d}k} e ^ {\ln \left( 1 - e ^ {-\frac{kn}{m}} \right) ^ k} \\
&= \dfrac{\mathrm{d}}{\mathrm{d}k} e ^ {k \ln \left( 1 - e ^ {-\frac{kn}{m}} \right)} \\
&= e ^ {k \ln \left( 1 - e ^ {-\frac{kn}{m}} \right)} \dfrac{\mathrm{d}}{\mathrm{d}k} \left[ k \ln \left( 1 - e ^ {-\frac{kn}{m}} \right) \right] \\
&= \left( 1 - e ^ {- \frac{kn}{m}} \right) ^ k \left[ \ln \left( 1 - e ^ {-\frac{kn}{m}} \right) + \frac{kn}{m} \frac{e ^ {-\frac{kn}{m}}}{\left(1 - e ^ {-\frac{kn}{m}} \right)}  \right]
\end{align*}
$$

$$
\begin{align*}
\left( 1 - e ^ {- \frac{kn}{m}} \right) ^ k \left[ \ln \left( 1 - e ^ {-\frac{kn}{m}} \right) + \frac{kn}{m} \frac{e ^ {-\frac{kn}{m}}}{\left(1 - e ^ {-\frac{kn}{m}} \right)}  \right] &= 0, \\
\ln \left( 1 - e ^ {-\frac{kn}{m}} \right) + \frac{kn}{m} \frac{e ^ {-\frac{kn}{m}}}{\left(1 - e ^ {-\frac{kn}{m}} \right)} &= 0, && \text{make substitution } b = \frac{m}{n} \\
\ln \left( 1 - e ^ {- \frac{k}{b}} \right) + k \frac{e ^ {- \frac{k}{b}}}{b \left( 1 - e ^ {- \frac{k}{b}} \right)} &= 0, && \text{make substitution } z = e ^ {- \frac{k}{b}},\ k = -b \ln z \\
\ln \left( 1 - y \right) - \frac{y \ln y}{1 - y} &= 0, \\
\left( 1 - y \right) \ln \left( 1 - y \right) &= y \ln y, \\
y &= \frac{1}{2}, \\
e ^ {- \frac{k}{b}} &= \frac{1}{2}, \\
- \frac{k}{b} & = \ln \frac{1}{2}, \\
\frac{k}{b} &= \ln 2, \\
k &= b \ln 2, \\
k &= \frac{m}{n} \ln 2.
\end{align*}
$$

## Optimal number of bits

Given a number of values to be inserted $n$ and a desired false positive probability $p$, we can compute required number of bits $m$

$$
\begin{align*}
p &= \left( 1 - e ^ {- \frac{kn}{m}} \right) ^ k, \\
p &= \left( 1 - e ^ {- \left( \frac{n}{m} \ln 2 \right) \frac{m}{n}} \right) ^ { \frac{m}{n} \ln 2}, \\
\ln p &= \ln \left( 1 - e ^ {- \left( \frac{n}{m} \ln 2 \right) \frac{m}{n}} \right) ^ { \frac{m}{n} \ln 2}, \\
\ln p &= - \frac{m}{n} \left( \ln 2 \right) ^ 2, \\
m &= - \frac{n \ln p}{\left( \ln 2 \right) ^ 2}.
\end{align*}
$$

## Implementation

In [8]:
import math
import mmh3

from bitarray import bitarray


class BloomFilter:

    def __init__(self, capacity, error_rate=0.1):
        self.capacity = capacity
        self.error_rate = error_rate
        self.nbits = math.ceil(-capacity * math.log(error_rate) / pow(math.log(2), 2))
        self.nhashes = round(math.log(2) * self.nbits / capacity)
        self.bits = bitarray(self.nbits)
        self.bits.setall(False)
        self.size = 0
    
    def _hashes(self, key):
        h1, h2 = mmh3.hash64(key, signed=False)
        combined_hash = h1
        for _ in range(self.nhashes):
            yield combined_hash & 0xffffffffffffffff
            combined_hash += h2
    
    def add(self, key):
        for h in self._hashes(key):
            self.bits[h % self.nbits] = True
        self.size += 1
    
    def has(self, key):
        return all(self.bits[h % self.nbits]
                   for h in self._hashes(key))
    
    def __contains__(self, key):
        return self.has(key)
    
    def __len__(self):
        return self.size


bf = BloomFilter(capacity=500000, error_rate=0.1)

with open('/usr/share/dict/american-english') as f:
    for w in f:
        bf.add(w.strip())

print(f'Number of bits: {bf.nbits}')
print(f'Number of hash functions: {bf.nhashes}')

words = (
    'hello',
    'hi',
    'hai',
    'linux',
    'python',
    'scar',
    'scary',
    'scarry')

for w in words:
    print('%r' % w, 'maybe present' if w in bf else 'is not present')

Number of bits: 2396265
Number of hash functions: 3
'hello' maybe present
'hi' maybe present
'hai' is not present
'linux' is not present
'python' maybe present
'scar' maybe present
'scary' maybe present
'scarry' is not present
